# ChatGLM3微调实战-LoRA技术微调

## 步骤1 导入相关包

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import os

# 配置环境变量
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'
os.environ['MODELSCOPE_CACHE']='/root/autodl-tmp/cache/'

#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('ZhipuAI/chatglm3-6b-base')

## 步骤2 加载数据集

In [ ]:
ds = load_dataset("llm-wizard/alpaca-gpt4-data-zh")
ds

In [ ]:
ds[:1]

## 步骤3 数据集预处理

In [ ]:
#加载本地模型，提前下载到本地
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/cache/modelscope/hub/ZhipuAI/chatglm3-6b-base", trust_remote_code=True)
tokenizer

In [ ]:
def process_func(example):
    MAX_LENGTH = 256 # 设置最大长度为256
    input_ids, attention_mask, labels = [], [], [] # 初始化输入ID、注意力掩码和标签列表
    instruction = "\n".join([example["instruction"], example["input"]]).strip()     # prompt
    instruction = tokenizer.build_chat_input(instruction, history=[], role="user")  # [gMASK]sop<|user|> \n prompt <|assistant|>
    response = tokenizer("\n" + example["output"], add_special_tokens=False)        # \n response
    input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id] #eos token
    attention_mask = instruction["attention_mask"][0].numpy().tolist() + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"][0].numpy().tolist()) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

In [ ]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

In [ ]:
# 检查数据（是否包含结束符）
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

## 步骤4 创建模型

In [ ]:
import torch
model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/cache/modelscope/hub/ZhipuAI/chatglm3-6b-base", 
                low_cpu_mem_usage=True,torch_dtype=torch.half,device_map="auto")
model.dtype

In [ ]:
for name, param in model.named_parameters():
    print(name)

### 1、PEFT 步骤1 配置文件

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

config = LoraConfig(target_modules=["query_key_value"])
config

### 2、PEFT 步骤2 创建模型

In [ ]:
model = get_peft_model(model, config)
model

In [ ]:
# 打印出模型中可训练参数的数量
model.print_trainable_parameters()

In [ ]:
# 查看模型参数，查看LoRA层添加到哪
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

## 步骤5 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/cache/finetuning/chatglm3-6b-base-lora", #输出目录，用于存储模型和日志文件。
    per_device_train_batch_size=2, # 每个设备的训练批次大小，即每个设备每次处理的数据量，批次越大，训练时需要资源越多
    gradient_accumulation_steps=16, # 指定梯度累积步数。用于控制梯度更新的频率。在每个累积步中，模型会计算多个批次的梯度，然后一次性更新权重。这可以减少内存占用并提高训练速度。在本例子中，每16个步骤进行一次梯度更新。
    logging_steps=10, #日志记录步数，用于控制每隔多少步记录一次训练日志。
    num_train_epochs=1, #训练轮数，即模型在整个训练集上进行迭代的次数。正常情况会训练很多轮
    learning_rate=1e-4, #学习率，控制模型参数更新的速度。较小的学习率会使模型收敛得更快，但可能需要更多的训练轮数
    adam_epsilon=1e-4, #Adam优化器的epsilon值，用于防止除以零的情况。
    remove_unused_columns=False #是否移除未使用的列，如果设置为False，则保留所有列，否则只保留模型所需的列。
)

## 步骤6 创建训练器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## 步骤7 模型训练

In [ ]:
trainer.train()

## 步骤8 模型推理

In [ ]:
model.eval()
print(model.chat(tokenizer, "如何写简历？", history=[])[0])